### About the Dataset:

###### 1. title : The title of a news article


###### 2. text : The text of the article
 

###### 3. subject : Subject of a news article
 

###### 4. date : date when the article was released
        

######     1 ---->Fake news

###### 0 ---->Real news

### Importing the Dependences

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
stopwords_list = stopwords.words("english")

### Data preprocessing

In [4]:
# Adding class feature to the Fake news data
Fake_news = pd.read_csv("fake.csv")
Fake_news["class"] = 1
Fake_news

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [5]:
# Adding class feature to the True news data
True_news = pd.read_csv("true.csv")
True_news["class"] = 0
True_news

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [6]:
# Concatenating the Fake news and True news data
news_data = pd.concat([Fake_news,True_news],axis=0)

In [7]:
# Loading the news_data dataset
news_data

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [8]:
news_data.shape

(44898, 5)

In [9]:
# print the first five rows of the dataFrame
news_data.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [10]:
# print the last five rows of the dataFrame
news_data.tail()

,title,text,subject,date,class
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [11]:
news_data.shape

(44898, 5)

In [12]:
# Getting some info about the data
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   class    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [13]:
# Checking the number of missing values in the data
news_data.isnull().sum()

title      0
text       0
subject    0
date       0
class      0
dtype: int64

In [14]:
# Checking the distribution of the class variable
news_data["class"].value_counts()

class
1    23481
0    21417
Name: count, dtype: int64

### 0 --> True News

### 1 --> Fake News

In [15]:
copy_news_data = news_data.copy()

In [16]:
# Merging the title and subject feature as content feature
copy_news_data["content"] = copy_news_data["title"]+' '+copy_news_data["subject"] 

In [17]:
copy_news_data["content"]

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorit...
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: content, Length: 44898, dtype: object

### Creating the Feature and Target variable

In [18]:
X = copy_news_data.drop(columns=["date","class"],axis=1)
y = copy_news_data["class"]

## Stemming

### Stemming is the process of reducing a word to its root word

### Examples : actor, actress, acting ---> act

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [21]:
copy_news_data["content"] = copy_news_data["content"].apply(stemming)

In [22]:
print(copy_news_data["content"])

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3        trump obsess even obama name code websit imag ...
4        pope franci call donald trump christma speech ...
                               ...                        
21412    fulli commit nato back new u approach afghanis...
21413    lexisnexi withdrew two product chines market w...
21414               minsk cultur hub becom author worldnew
21415    vatican upbeat possibl pope franci visit russi...
21416     indonesia buy billion worth russian jet worldnew
Name: content, Length: 44898, dtype: object


In [23]:
# Again creating the Feature and Target variable
X = copy_news_data["content"].values
y = copy_news_data["class"].values

In [24]:
X

array(['donald trump send embarrass new year eve messag disturb news',
       'drunk brag trump staffer start russian collus investig news',
       'sheriff david clark becom internet joke threaten poke peopl eye news',
       ..., 'minsk cultur hub becom author worldnew',
       'vatican upbeat possibl pope franci visit russia worldnew',
       'indonesia buy billion worth russian jet worldnew'], dtype=object)

In [25]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [26]:
y.shape

(44898,)

In [27]:
# Converting the textual data to numeric data
vertorizer = TfidfVectorizer()
vertorizer.fit(X)
X = vertorizer.transform(X)

In [28]:
print(X)

  (0, 13106)	0.2841233635673885
  (0, 12011)	0.11801151210378362
  (0, 10342)	0.3539654263616356
  (0, 7874)	0.11122516508183175
  (0, 7866)	0.23407158318916954
  (0, 7335)	0.3484551980847141
  (0, 3949)	0.46447759682434775
  (0, 3731)	0.3635875781057666
  (0, 3404)	0.2705922497170223
  (0, 3339)	0.4126967109620307
  (1, 12011)	0.11624188294592752
  (1, 11066)	0.3360943239614603
  (1, 11032)	0.40270520669068827
  (1, 9997)	0.2787247463882496
  (1, 7874)	0.1095572998735369
  (1, 5989)	0.31084230957734127
  (1, 3523)	0.451785166223473
  (1, 2289)	0.40892456234396
  (1, 1414)	0.39206451614289717
  (2, 11718)	0.25092146173015395
  (2, 10473)	0.30794879739595504
  (2, 8836)	0.42306308562413203
  (2, 8601)	0.23286014723695025
  (2, 7874)	0.08851503079061332
  (2, 6194)	0.31676212135890697
  :	:
  (44894, 9068)	0.38418700982994514
  (44894, 7114)	0.3383825860096951
  (44894, 6698)	0.5384575482105356
  (44894, 2037)	0.3383825860096951
  (44895, 13018)	0.13294438655412533
  (44895, 7446)	0.5519

### Splitting the data into Training data and Test data

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,stratify= y,random_state=150)

In [30]:
print(X_train)

  (0, 12115)	0.3573482939981762
  (0, 12011)	0.14082529277177583
  (0, 8856)	0.156870317641997
  (0, 7501)	0.5267590558861619
  (0, 7215)	0.4565856565241056
  (0, 690)	0.5847363272737606
  (1, 12944)	0.25047449345953965
  (1, 11548)	0.46833195234540226
  (1, 11153)	0.35273018602981765
  (1, 8856)	0.11146580107728085
  (1, 8833)	0.39744329411166446
  (1, 6888)	0.3790977051241261
  (1, 4670)	0.3572622685702533
  (1, 2792)	0.2726304897224664
  (1, 1112)	0.2782886038937199
  (2, 12740)	0.27376334012876585
  (2, 8856)	0.13085183798336988
  (2, 7603)	0.536494733189231
  (2, 6580)	0.3029980885750831
  (2, 4397)	0.39041231180732305
  (2, 2753)	0.36224112281109483
  (2, 2331)	0.4946277240630535
  (3, 13106)	0.33172916861117613
  (3, 9351)	0.40742578611573377
  (3, 9001)	0.26882595869852965
  :	:
  (31425, 9050)	0.3399758100927116
  (31425, 8879)	0.32279479015708157
  (31425, 8850)	0.1199079528188555
  (31425, 8552)	0.2628342786070448
  (31425, 4003)	0.3494312076767966
  (31425, 2283)	0.26249255

In [31]:
y_train

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

### Training the model using Logistic Regression

In [33]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

### Model Evaluation using accuracy score


In [34]:
# Training data accuracy
X_train_prediction = model.predict(X_train)
train_data_accuracy = accuracy_score(y_train,X_train_prediction)

In [35]:
print("Accuracy of the training data :",train_data_accuracy)

Accuracy of the training data : 0.9999681812396589


In [36]:
# Test data accuracy
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test,X_test_prediction)

In [37]:
print("Accuracy of the test data :",test_data_accuracy)

Accuracy of the test data : 0.9998515219005196


### Building the Detection System

In [45]:
X_test[4399]

<1x13208 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [39]:
prediction = model.predict(X_test[4399])
print(prediction)

if (prediction[0]==0):
    print("The news is Real")
else:
    print("The news is Fake")

[0]
The news is Real


In [40]:
y_test[4399]

0

In [41]:
import pickle as pkl
filename = 'classifier.pkl'
pkl.dump(model,open(filename,'wb'))

In [42]:
news_data.to_csv('News_data.csv',index=False)

In [43]:
import pickle as pkl
filename = 'stopwords.pkl'
pkl.dump(stopwords_list,open(filename,'wb'))

In [46]:
import pickle as pkl
filename = 'vectorizer.pkl'
pkl.dump(vertorizer,open(filename,'wb'))